In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os

In [2]:
cd ../

D:\git_repos\lab_data_science


In [3]:
#TRAIN - VAL

In [4]:
import src.config as cfg
from src.utils import save_as_pickle, load_as_pickle

In [5]:
# raw_path = 'data/raw/'
# train = pd.read_csv(os.path.join(raw_path, 'train.csv'))
# test = pd.read_csv(os.path.join(raw_path, 'test.csv'))

In [6]:
# TARGET_COLS = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']
# ID_COL = 'ID'
# EDU_COL = 'Образование'
# SEX_COL = 'Пол'
# CAT_COLS = [
#     'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 
#     'Профессия', 'Статус Курения', 'Алкоголь',
#     'Время засыпания', 'Время пробуждения'
# ]
# OHE_COLS = [
#     'Пол', 'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
#     'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма', 'Туберкулез легких ', 'ВИЧ/СПИД',
#     'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы','Пассивное курение', 'Сон после обеда', 
#     'Спорт, клубы', 'Религия, клубы'
# ]
# REAL_COLS = ['Возраст курения', 'Сигарет в день', 'Возраст алког', 'Частота пасс кур']

In [7]:
# def set_idx(df: pd.DataFrame, idx_col: str) -> pd.DataFrame:
#     df = df.set_index(idx_col)
#     return df

In [8]:
# train = train.set_index(ID_COL)
# test = test.set_index(ID_COL)

In [9]:
# train, target = train.drop(TARGET_COLS, axis=1), train[TARGET_COLS]

In [10]:
# train.pipe()

In [11]:
# def preprocess(df: pd.DataFrame) -> pd.DataFrame:
#     df = set_idx(df, ID_COL)
#     df = drop_unnecesary_id(df)
#     df = fill_sex(df)
#     df = cast_types(df)
#     return df
    
    

In [12]:
# def drop_unnecesary_id(df: pd.DataFrame) -> pd.DataFrame:
#     if 'ID_y' in df.columns:
#         df = df.drop('ID_y', axis=1)
#     return df

In [13]:
# train = drop_unnecesary_id(train)

In [14]:
# def add_ord_edu(df: pd.DataFrame) -> pd.DataFrame:
#     df[f'{EDU_COL}_ord'] = df[EDU_COL].str.slice(0, 1).astype(np.int8).values
#     return df

In [15]:
# train = add_ord_edu(train)
# test = add_ord_edu(test)

In [16]:
# def fill_sex(df: pd.DataFrame) -> pd.DataFrame:
#     most_freq = df[SEX_COL].value_counts().index[0]
#     df[SEX_COL] = df[SEX_COL].fillna(most_freq)
#     return df

In [17]:
# def cast_types(df: pd.DataFrame) -> pd.DataFrame:
#     df[CAT_COLS] = df[CAT_COLS].astype('category')

#     ohe_int_cols = train[OHE_COLS].select_dtypes('number').columns
#     df[ohe_int_cols] = df[ohe_int_cols].astype(np.int8)

#     df[REAL_COLS] = df[REAL_COLS].astype(np.float32)
#     return df
    

In [18]:
from sklearn.metrics import r2_score, mean_squared_error, precision_score, recall_score, roc_auc_score, f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [19]:
train_data_path = 'data/interim/train.pkl'
target_data_path = 'data/interim/target.pkl'

In [20]:
def load_splitted_data(train_path, target_path):
    train = load_as_pickle(train_path)
    target = load_as_pickle(target_path)
    return train, target

In [21]:
train_data, target_data = load_splitted_data(train_data_path, target_data_path)

In [22]:
x_train, x_test, y_train, y_test = train_test_split(train_data, target_data, test_size=0.25)

In [23]:
path_to_splitted_train_data = 'data/processed/train.pkl'
path_to_splitted_train_data_target = 'data/processed/train_target.pkl'
path_to_splitted_val_data = 'data/processed/val.pkl'
path_to_splitted_val_data_target = 'data/processed/val_target.pkl'

In [24]:
save_as_pickle(x_train, path_to_splitted_train_data)
save_as_pickle(y_train, path_to_splitted_train_data_target)
save_as_pickle(x_test, path_to_splitted_val_data)
save_as_pickle(y_test, path_to_splitted_val_data_target)

In [25]:
#FEATURE GENERATION

In [26]:
import category_encoders as ce

In [27]:
def second_hand_smoke_count(value):
    if value == '1-2 раза в неделю':
        return 1.5
    if value == '2-3 раза в день':
        return 17.5
    if value == '3-6 раз в неделю':
        return 4.5
    if value == '4 и более раз в день':
        return 28.0
    if value == 'не менее 1 раза в день':
        return 7.0
    return 0.0

In [28]:
def fill_real_cols_with_zero(df):
    df = df.fillna(0.)
    return df

In [29]:
def fill_real_cols(data):
    data.loc[:, cfg.REAL_COLS[:-1]] = fill_real_cols_with_zero(data.loc[:, cfg.REAL_COLS[:-1]])
    data['Частота пасс кур'] = [second_hand_smoke_count(i) for i in data['Частота пасс кур']]
    return data

In [30]:
x_train = fill_real_cols(x_train)

In [31]:
x_train.shape, y_train.shape

((716, 32), (716, 5))

In [32]:
# def target_encode_multiclass(X,y): #X,y are pandas df and series
#     y=y.astype(str)  #convert to string to onehot encode
#     enc=ce.OneHotEncoder().fit(y)
#     y_onehot=enc.transform(y) 

#     class_names=y_onehot.columns  #names of onehot encoded columns

#     X_obj=X.select_dtypes('object') #separate categorical columns
#     X=X.select_dtypes(exclude='object')

#     for class_ in class_names:

#         enc=ce.TargetEncoder()
#         enc.fit(X_obj,y_onehot[class_]) #convert all categorical
#         temp=enc.transform(X_obj)       #columns for class_
#         temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
#         X=pd.concat([X,temp],axis=1)    #add to original dataset

#     return X

In [33]:
# te = TargetEncoder(verbose=5, cols = cfg.CAT_COLS, drop_invariant=True, return_df=True)
# target_df_train = target_encode_multiclass(x_train.loc[:, cfg.CAT_COLS], y_train)
# target_df_train

In [154]:
# te = [ce.TargetEncoder(verbose=5, drop_invariant=False)] * 5
# te[0].fit_transform(x_train[cfg.CAT_COLS], y_train[y_train.columns[0]])

C:\git\lab_data_science\venv\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\git\lab_data_science\venv\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Статус Курения,Алкоголь,Время засыпания,Время пробуждения
ID,,,,,,,,,,,
54-002-130-01,0.352113,0.430622,0.461207,0.459138,0.477462,0.522124,0.449275,0.492857,0.417722,0.420000,0.456044
54-001-103-01,0.352113,0.430622,0.461207,0.459138,0.349206,0.465278,0.285714,0.335294,0.417722,0.514925,0.444444
54-102-199-02,0.504970,0.377049,0.461207,0.459138,0.477462,0.358079,0.420118,0.335294,0.417722,0.457265,0.428571
54-501-082-01,0.504970,0.430622,0.461207,0.459138,0.477462,0.465278,0.449275,0.500000,0.417722,0.457265,0.572917
54-102-097-01,0.504970,0.688073,0.461207,0.459138,0.477462,0.522124,0.620690,0.500000,0.543103,0.476821,0.456044
...,...,...,...,...,...,...,...,...,...,...,...
54-001-006-03,0.504970,0.430622,0.461207,0.459138,0.477462,0.465278,0.620690,0.492857,0.417722,0.457265,0.429348
54-501-044-02,0.352113,0.430622,0.461207,0.459138,0.477462,0.522124,0.449275,0.492857,0.417722,0.514925,0.620690
54-602-035-02,0.352113,0.430622,0.461207,0.459138,0.477462,0.358079,0.536232,0.492857,0.417722,0.457265,0.456044


In [172]:
num_classes = 5
te = [ce.TargetEncoder(drop_invariant=False)] * num_classes

C:\git\lab_data_science\venv\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\git\lab_data_science\venv\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [178]:
def target_encoding(cat_features, x_train, y_train=None, test=False):
    target_encoded_x_train = x_train.copy()
    for target_num in range(num_classes):
        target_col = cfg.TARGET_COLS[target_num]
        if test:
            encoded_cols = te[target_num].transform(x_train[cat_features])
        else:
            encoded_cols = te[target_num].fit_transform(x_train[cat_features], y_train[target_col])
            
        target_encoded_x_train[[col + "/" + target_col for col in cfg.CAT_COLS]] = encoded_cols

    target_encoded_x_train = target_encoded_x_train.drop(cat_features, axis=1)
    return target_encoded_x_train
#     target_encoded_x_train.insert(loc=target_encoded_x_train.columns[:-1],
#                                   column=[col + "/" + target_col for col in cfg.CAT_COLS],
#                                   value=encoded_cols,
#                                   allow_duplicates = False)
        

In [179]:
target_encoded_x_train = target_encoding(cfg.CAT_COLS, x_train, y_train)
target_encoded_x_train

,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,...,Семья/Прочие заболевания сердца,Этнос/Прочие заболевания сердца,Национальность/Прочие заболевания сердца,Религия/Прочие заболевания сердца,Образование/Прочие заболевания сердца,Профессия/Прочие заболевания сердца,Статус Курения/Прочие заболевания сердца,Алкоголь/Прочие заболевания сердца,Время засыпания/Прочие заболевания сердца,Время пробуждения/Прочие заболевания сердца
ID,,,,,,,,,,,,,,,,,,,,,
54-002-130-01,0,0,0,0,0,0,1,0,0,0,...,0.076555,0.087644,0.086181,0.095159,0.094395,0.101449,0.078571,0.078059,0.060000,0.076923
54-001-103-01,1,0,0,0,0,0,0,0,0,0,...,0.076555,0.087644,0.086181,0.047619,0.055556,0.000000,0.082353,0.078059,0.104478,0.083333
54-102-199-02,1,0,0,0,0,0,0,0,0,0,...,0.098361,0.087644,0.086181,0.095159,0.091703,0.124260,0.082353,0.078059,0.055556,0.190476
54-501-082-01,0,1,0,1,0,0,0,0,0,0,...,0.076555,0.087644,0.086181,0.095159,0.055556,0.101449,0.088670,0.078059,0.055556,0.062500
54-102-097-01,0,1,0,1,0,1,0,0,0,0,...,0.100917,0.087644,0.086181,0.095159,0.094395,0.086207,0.088670,0.068966,0.086093,0.076923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54-001-006-03,0,1,0,0,0,0,0,0,0,0,...,0.076555,0.087644,0.086181,0.095159,0.055556,0.086207,0.078571,0.078059,0.055556,0.059783
54-501-044-02,1,0,0,0,0,0,0,0,0,0,...,0.076555,0.087644,0.086181,0.095159,0.094395,0.101449,0.078571,0.078059,0.104478,0.068966
54-602-035-02,1,0,0,0,0,0,0,0,0,0,...,0.076555,0.087644,0.086181,0.095159,0.091703,0.072464,0.078571,0.078059,0.055556,0.076923


In [ ]:
train_data_for_catboost, 

In [ ]:
#MODELS

In [ ]:
import catboost

In [ ]:
def get_indexes_of_cat_columns(df: pd.DataFrame, col_names: str):
    idx = []
    for col in col_names:
        idx.append(df.columns.tolist().index(col))
        
    return idx

In [ ]:
cfg.CAT_COLS

In [ ]:
get_indexes_of_cat_columns(train_data, cfg.CAT_COLS)

In [ ]:
cat_idx = get_indexes_of_cat_columns(train_data, cfg.CAT_COLS)
cat_idx

In [ ]:
# for i in range(100, 500, 100):
i = 100
boosting_model = catboost.CatBoostClassifier(
    loss_function='MultiLogloss',
    cat_features=cat_idx
)
    

In [ ]:
#Так как крайне важно правильно предсказать наличие смертельной болезни, для нашей задачи хорошо подойдет метрика recall, показывающая долю верно предсказанных болезней. Также, чтобы выявить количество ложно положительных результатов(то есть случаев, когда модель выявила болезнь у здорового человека), можно использовать метрику presicion. 

In [ ]:
# train_pool = catboost.Pool(x_train, y_train)
# val_pool = catboost.Pool(x_test, y_test)

# boosting_model.fit(train_data, target_data)
boosting_model = boosting_model.fit(x_train, y_train, metric_period=10, verbose=50)

In [ ]:
preds = boosting_model.predict(x_test)

In [188]:
def count_metrics(y_test, preds):
    print(precision_score(y_test, preds, average='micro'))
    print(recall_score(y_test, preds, average='micro'))
    print(f1_score(y_test, preds, average='micro'))
    print(roc_auc_score(y_test, preds, average='micro'))

In [137]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [190]:
fCl_target_encode = MultiOutputClassifier(LogisticRegression()).fit(target_encoded_x_train, y_train)
# fCl = MultiOutputClassifier(RandomForestClassifier()).fit(x_train, y_train)

C:\git\lab_data_science\venv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\git\lab_data_science\venv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

In [191]:
target_encoded_test = target_encoding(cfg.CAT_COLS, x_test, test=True)
target_encoded_test = fill_real_cols(target_encoded_test)
preds = fCl_target_encode.predict(target_encoded_test)

In [192]:
count_metrics(y_test, preds)

0.5362318840579711
0.1778846153846154
0.26714801444043323
0.5727315680773127
